In [1]:
import boto3
import os
from botocore.exceptions import NoCredentialsError, EndpointConnectionError


In [ ]:

def get_s3_files(bucket_name, s3_folder, region_name):
    """Lấy danh sách tất cả file trong folder trên S3."""
    s3 = boto3.client('s3', region_name=region_name)
    s3_files = []
    continuation_token = None

    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder)

        if 'Contents' in response:
            s3_files.extend([obj['Key'] for obj in response['Contents']])

        if response.get('IsTruncated'):  # Có thêm dữ liệu
            continuation_token = response['NextContinuationToken']
        else:
            break

    return s3_files

def get_local_files(local_folder):
    """Lấy danh sách tất cả file trong folder trên máy tính."""
    local_files = []
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            # Lưu đường dẫn file tương đối từ folder gốc
            relative_path = os.path.relpath(os.path.join(root, file), local_folder).replace("\\", "/")
            local_files.append(relative_path)
    return local_files

def upload_files_to_s3(local_folder, bucket_name, s3_folder, region_name):
    try:
        # Khởi tạo client S3
        s3 = boto3.client('s3', region_name=region_name)

        # Lấy danh sách file trên S3 và máy tính
        s3_files = get_s3_files(bucket_name, s3_folder, region_name)
        local_files = get_local_files(local_folder)

        # Tìm các file cục bộ chưa có trên S3
        files_to_upload = [file for file in local_files if f"{s3_folder}{file}" not in s3_files]

        if not files_to_upload:
            print("Tất cả file đã tồn tại trên S3. Không có file nào cần upload.")
            return

        # Upload các file chưa có với Content-Type là image/jpeg
        for file in files_to_upload:
            local_file_path = os.path.join(local_folder, file)
            s3_file_path = f"{s3_folder}{file}"

            # Upload file với Content-Type
            s3.upload_file(
                local_file_path,
                bucket_name,
                s3_file_path,
                ExtraArgs={'ContentType': 'image/jpeg'}
            )
            print(f"Uploaded: {local_file_path} to s3://{bucket_name}/{s3_file_path} with Content-Type=image/jpeg")

        print("Hoàn tất upload các file chưa có.")

    except NoCredentialsError:
        print("Không tìm thấy thông tin xác thực AWS. Hãy kiểm tra lại.")
    except EndpointConnectionError as e:
        print(f"Không thể kết nối đến endpoint: {e}")
    except Exception as e:
        print(f"Lỗi khác: {e}")

# Cấu hình thông tin
local_folder = "C:/path/to/your/local/folder"  # Thay thế bằng đường dẫn thư mục trên máy tính
bucket_name = "hs-search-img-pdf"             # Tên bucket S3
s3_folder = "Eximhow_img/"                    # Thư mục trên S3
region_name = "ap-southeast-1"                # Region của bucket

# Gọi hàm upload
upload_files_to_s3(local_folder, bucket_name, s3_folder, region_name)
